In [1]:
import requests
import urllib3
import streamlit as st
import pandas as pd
import warnings
import numpy as np
import supabase as sb

from meteostat import Point, Hourly, units
from concurrent.futures import ThreadPoolExecutor

warnings.simplefilter(action='ignore', category=FutureWarning)

##### STRAVA API DATA EXTRACTION ####
# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = 'https://www.strava.com/oauth/token'

payload = {
    'client_id': st.secrets['client_id'],
    'client_secret': st.secrets['client_secret'],
    'refresh_token': st.secrets['refresh_token'],
    'grant_type': 'refresh_token',
    'f': 'json'
}

res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + access_token}

def get_strava_data() -> pd.DataFrame:
    '''This function builds the dataframe from Strava API data. It is used to then cache the dataframe for faster loading in the Streamlit app.
    
    Returns:
        pre_df (DataFrame): DataFrame of activities and gear data'''
    
        
        # Strava API only allows 200 results per page. This function loops through until all results are collected
    def get_activities_data() -> pd.DataFrame:
        '''This function gets all activities data from Strava API
        
        Returns:
            data (DataFrame): Normalized JSON data of activities'''
            
        # set the URL for the Strava API
        activities_url = 'https://www.strava.com/api/v3/athlete/activities'
        # set value of page to start at page 1
        page = 1
        # create an empty list to store all data
        data = []
        # set new_results to True to start the loop
        new_results = True
        
        while new_results:
            # requests one page at a time (200 results)
            get_activities = requests.get(activities_url, headers=header, params={'per_page': 200, 'page': page}).json()
            # feedback
            print(f"Fetching page {page}")
            print(f"Number of activities fetched: {len(get_activities)}")
            # if there are no results, the loop will stop
            new_results = get_activities
            # add the results to the data list
            data.extend(get_activities)
            # increment the page number
            page += 1

            if page > 20:
                print('Stopping after 20 pages to avoid excessive API calls.')
                
                return pd.DataFrame()
            
        return pd.json_normalize(data)
            
    # get all activities data
    activities = get_activities_data()
    
    if activities.empty: 
        
        return get_data_from_database()
    
    # convert meters to miles
    activities.distance = (activities.distance / 1609.34).round(2)
    # convert to mph
    activities.average_speed = (activities.average_speed * 2.23694).round(2)
    activities.max_speed = (activities.max_speed * 2.23694).round(2)
    # convert to feet
    activities.total_elevation_gain = (activities.total_elevation_gain * 3.28084).round(2)
    activities.elev_high = (activities.elev_high * 3.28084).round(2)
    activities.elev_low = (activities.elev_low * 3.28084).round(2)

    activities_df = pd.DataFrame(activities)
    
    def add_weather_data(df: pd.DataFrame, max_workers=30) -> pd.DataFrame:
        '''This function gets weather data from Meteostat and adds it onto the activities DataFrame
        
        Args:
            df (DataFrame): Activities data frame that uses latitude, longitude, and timestamps to get weather data
            max_worker (int): Number of threads to use in the multi-threading process
            
        Returns:
            df (DataFrame): Original df with weatehr data appended'''
            
        def get_weather(row):
            '''This function takes the latitude, longitude, and timestamp for each row and calls the Meteostat API for data
            
            Args:
                row: The row in the DataFrame used in the parent function
                
            Returns:
                weather_data (dict): The temperature and relative humidity of the row's activity as a dictionary'''
            
            # get the location of the activity
            location = Point(row['start_latitude'], row['start_longitude'])
            # get the time of the activity
            timestamp = pd.to_datetime(row['start_date_local'])
            # only use the hour it started
            start = end = timestamp.replace(tzinfo=None, minute=0, second=0, microsecond=0)

            # call meteostat API
            try:
                data = Hourly(location, start, end)
                data = data.convert(units.imperial).fetch()
                if not data.empty:
                    # only get the first row of data
                    weather = data[['temp', 'rhum']].iloc[0]
                    return {'temp': weather['temp'], 'rhum': weather['rhum']}
                else:
                    return {'temp': None, 'rhum': None}
            except Exception as e:
                print(f"Error fetching weather for {timestamp}: {e}")
                return {'temp': None, 'rhum': None}
            
        # separate the latitude and longitude from the activity data
        df[['start_latitude', 'start_longitude']] = pd.DataFrame(df['start_latlng'].tolist(), index=df.index)
        
        # Prepare rows
        rows = [row for _, row in df.iterrows()]
        max_rows = len(rows)

        # multi-threading so the function can call the API and iterate through rows faster
        weather_data = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for i, result in enumerate(executor.map(get_weather, rows)):
                weather_data.append(result)

        # get the weatehr data and concat the two DataFrames
        weather_df = pd.DataFrame(weather_data)
        Hourly.clear_cache()
        return pd.concat([df.reset_index(drop=True), weather_df.reset_index(drop=True)], axis=1)
    
    activities_df = add_weather_data(activities_df)

    # get distinct gear id's
    gear_id_list = activities_df['gear_id'].unique()
    gear_id_list = gear_id_list[~pd.isnull(gear_id_list)]

    def get_gear_data(gear_list: list) -> pd.DataFrame:
        '''This function gets gear data from Strava API
        
        Args:
            gear_list (array): List of distinct gear ids
            
            Returns:
                data (DataFrame): Normalized JSON data of gear'''
        # set the URL for the Strava API
        gear_url = 'https://www.strava.com/api/v3/gear/{id}'
        # create empty list to store gear data
        data = []
        # loop through gear_list and get gear data
        for gear_id in gear_list:
            get_gear = requests.get(gear_url.format(id=gear_id), headers=header).json()
            data.append(get_gear)
        return pd.json_normalize(data)
    
    # get all gear data
    gear = get_gear_data(gear_id_list)

    # convert meters to miles
    gear.distance = gear.distance / 1609.34

    gear = gear.drop(columns=['converted_distance'])

    ##### DATA CLEANING AND TRANSFORMATION #####
    # create base dataframe joining activity and gear data
    pre_df = pd.merge(activities_df,
                    gear, 
                    how='left',
                    left_on='gear_id',
                    right_on='id',
                    suffixes=('_activity', '_gear')).drop(columns='id_gear')

    # convert moving_time and elapsed time to H% M% S% format
    pre_df['moving_time'] = pd.to_timedelta(pd.to_datetime(pre_df['moving_time'], unit='s').dt.strftime('%H:%M:%S'))
    pre_df['elapsed_time'] = pd.to_timedelta(pd.to_datetime(pre_df['elapsed_time'], unit='s').dt.strftime('%H:%M:%S'))

    # convert start_date and start_date_local to datetime
    pre_df['start_date'] = pd.to_datetime(pd.to_datetime(pre_df['start_date']).dt.strftime('%Y-%m-%d %H:%M:%S'))
    pre_df['start_date_local'] = pd.to_datetime(pd.to_datetime(pre_df['start_date_local']).dt.strftime('%Y-%m-%d %H:%M:%S'))

    # add start time for analysis and in am/pm format
    pre_df['start_time_local_24h'] = pd.to_datetime(pre_df['start_date_local']).dt.time
    pre_df['start_time_local_24h_hour'] = pd.to_datetime(pre_df['start_date_local']).dt.round('H').dt.hour
    pre_df['start_time_local_12h'] = pd.to_datetime(pre_df['start_date_local']).dt.strftime("%I:%M %p")

    # add day of week
    pre_df['weekday'] = pd.to_datetime(pre_df['start_date_local']).dt.day_name()
    pre_df['weekday_num'] = pd.to_datetime(pre_df['start_date_local']).dt.weekday

    # add month
    pre_df['month'] = pd.to_datetime(pre_df['start_date_local']).dt.month_name()
    pre_df['month_num'] = pd.to_datetime(pre_df['start_date_local']).dt.month
    pre_df['monthly_date'] = pd.to_datetime(pd.to_datetime(pre_df['start_date_local']).dt.strftime('%Y-%m')).apply(lambda x: x.replace(year=2025))

    # add month year
    pre_df['month_year'] = pd.to_datetime(pd.to_datetime(pre_df['start_date_local']).dt.strftime('%Y-%m'))
    
    # add month year name
    pre_df['month_year_name'] = pd.to_datetime(pre_df['start_date_local']).dt.strftime('%b %Y')

    # add year label
    pre_df['year'] = pd.to_datetime(pre_df['start_date_local']).dt.year
    
    #add timestamp
    pre_df['refresh_date'] = pd.Timestamp.now(tz='America/New_York').strftime('%Y-%m-%d %I:%M %p')
    
    pre_df.drop(columns=['start_latlng', 'end_latlng', 'start_latitude', 'start_longitude'], inplace=True)
    
    return pre_df
    
def get_data_from_database() -> pd.DataFrame:
    '''This function calls the supabase table that holds the Strava API data
    
    Returns:
        df (DataFrame): Dataframe loaded from table'''
        
    url = st.secrets['supabase_url']
    key = st.secrets['supabase_secret']

    supabase = sb.create_client(url, key)

    # get all data from the table
    response = supabase.table("tom_runs_the_world").select("*").execute()
    df = pd.DataFrame(response.data)
    
    if df.empty:
        df = get_strava_data()
        send_data_to_database(df)
    
    # convert timedeltas
    df['moving_time'] = pd.to_timedelta(df['moving_time'])
    df['elapsed_time'] = pd.to_timedelta(df['elapsed_time'])
    # convert time object
    df['start_time_local_24h'] = pd.to_datetime(df['start_time_local_24h']).dt.time
    # convert datetime
    df['start_date'] = pd.to_datetime(pd.to_datetime(df['start_date']).dt.strftime('%Y-%m-%d %H:%M:%S'))
    df['start_date_local'] = pd.to_datetime(pd.to_datetime(df['start_date_local']).dt.strftime('%Y-%m-%d %H:%M:%S'))
    df['monthly_date'] = pd.to_datetime(pd.to_datetime(df['start_date_local']).dt.strftime('%Y-%m'))
    df['month_year'] = pd.to_datetime(pd.to_datetime(df['start_date_local']).dt.strftime('%Y-%m'))
    
    return df

def send_data_to_database(df: pd.DataFrame) -> None:
    '''This function replaces the data in the database table with the new Strava API data
    
    Args:
        df (DataFrame): DataFrame to send to the table'''
       
    # connect to supabase
    url = st.secrets['supabase_url']
    key = st.secrets['supabase_secret']

    supabase = sb.create_client(url, key)
    
    # Delete all data
    supabase.table("tom_runs_the_world").delete().neq("id_activity", 0).execute()

    # convert timedeltas
    df['moving_time'] = df['moving_time'].astype(str)
    df['elapsed_time'] = df['elapsed_time'].astype(str)
    # convert time object
    df['start_time_local_24h'] = df['start_time_local_24h'].apply(lambda x: x.strftime('%H:%M:%S'))
    # convert datetime
    df['start_date'] = df['start_date'].apply(lambda x: x.isoformat())
    df['start_date_local'] = df['start_date_local'].apply(lambda x: x.isoformat())
    df['monthly_date'] = df['monthly_date'].apply(lambda x: x.isoformat())
    df['month_year'] = df['month_year'].apply(lambda x: x.isoformat())

    # Replace np.nan with None for any other remaining NaN values
    df_clean = df.replace({np.nan: None})

    # Convert the cleaned DataFrame to a list of dictionaries
    records = df_clean.to_dict(orient='records')
    
    # insert new data
    supabase.table("tom_runs_the_world").insert(records).execute()

In [2]:
df = get_strava_data()

Fetching page 1
Number of activities fetched: 200
Fetching page 2
Number of activities fetched: 200
Fetching page 3
Number of activities fetched: 200
Fetching page 4
Number of activities fetched: 47
Fetching page 5
Number of activities fetched: 0
Error fetching weather for 2025-05-20 19:45:12+00:00: Ran out of input
Error fetching weather for 2025-06-07 17:59:23+00:00: Ran out of input
Error fetching weather for 2025-05-31 10:02:41+00:00: pickle data was truncated
Error fetching weather for 2025-04-18 19:32:51+00:00: Ran out of input
Error fetching weather for 2025-05-26 20:11:56+00:00: pickle data was truncated
Error fetching weather for 2025-05-17 12:34:34+00:00: pickle data was truncated
Error fetching weather for 2025-03-29 09:49:56+00:00: invalid load key, '\x00'.
Error fetching weather for 2025-03-30 18:54:10+00:00: invalid load key, '\x00'.
Error fetching weather for 2025-04-04 17:38:29+00:00: invalid load key, '\x00'.
Error fetching weather for 2025-03-12 19:21:29+00:00: pickle

In [3]:
df

,resource_state_activity,name_activity,distance_activity,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id_activity,...,start_time_local_12h,weekday,weekday_num,month,month_num,monthly_date,month_year,month_year_name,year,refresh_date
0,2,Afternoon Run,6.01,0 days 01:05:51,0 days 01:06:15,253.94,Run,Run,NaN,14728433111,...,05:59 PM,Saturday,5,June,6,2025-06-01,2025-06-01,Jun 2025,2025,2025-06-09 10:16 PM
1,2,Evening Walk,1.94,0 days 00:38:16,0 days 00:38:16,112.20,Walk,Walk,NaN,14678838548,...,07:19 PM,Monday,0,June,6,2025-06-01,2025-06-01,Jun 2025,2025,2025-06-09 10:16 PM
2,2,Afternoon Run,8.17,0 days 01:21:10,0 days 01:21:38,282.81,Run,Run,NaN,14667796164,...,05:28 PM,Sunday,6,June,6,2025-06-01,2025-06-01,Jun 2025,2025,2025-06-09 10:16 PM
3,2,Afternoon Walk,1.44,0 days 00:38:48,0 days 00:41:37,342.52,Walk,Walk,NaN,14667796328,...,03:49 PM,Sunday,6,June,6,2025-06-01,2025-06-01,Jun 2025,2025,2025-06-09 10:16 PM
4,2,Morning Jaunt,4.61,0 days 01:33:51,0 days 01:34:35,316.27,Walk,Walk,NaN,14656862907,...,10:02 AM,Saturday,5,May,5,2025-05-01,2025-05-01,May 2025,2025,2025-06-09 10:16 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2,Afternoon Run,0.98,0 days 00:08:32,0 days 00:08:48,54.79,Run,Run,NaN,3903866794,...,05:02 PM,Friday,4,August,8,2025-08-01,2020-08-01,Aug 2020,2020,2025-06-09 10:16 PM
643,2,Evening Run,1.01,0 days 00:10:53,0 days 00:10:56,201.12,Run,Run,NaN,3903866817,...,06:41 PM,Thursday,3,August,8,2025-08-01,2020-08-01,Aug 2020,2020,2025-06-09 10:16 PM
644,2,Evening Run,1.02,0 days 00:09:25,0 days 00:09:25,51.51,Run,Run,NaN,3903866790,...,06:38 PM,Friday,4,July,7,2025-07-01,2020-07-01,Jul 2020,2020,2025-06-09 10:16 PM
645,2,Morning Walk,1.26,0 days 00:22:22,0 days 00:22:22,30.18,Walk,Walk,NaN,3391765082,...,10:42 AM,Sunday,6,May,5,2025-05-01,2020-05-01,May 2020,2020,2025-06-09 10:16 PM


In [16]:
send_data_to_database(df)

AttributeError: 'str' object has no attribute 'strftime'

In [14]:
df_test = get_data_from_database()

In [15]:
df_test

,resource_state_activity,name_activity,distance_activity,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id_activity,...,start_time_local_12h,weekday,weekday_num,month,month_num,monthly_date,month_year,month_year_name,year,refresh_date
0,2,Morning Jaunt,4.61,0 days 01:33:51,0 days 01:34:35,316.27,Walk,Walk,None,14656862907,...,10:02:00,Saturday,5,May,5,2025-05-01,2025-05-01,May 2025,2025,2025-05-31T21:31:00
1,2,Afternoon Run,4.02,0 days 00:40:36,0 days 00:42:30,168.64,Run,Run,None,14646346903,...,17:30:00,Friday,4,May,5,2025-05-01,2025-05-01,May 2025,2025,2025-05-31T21:31:00
2,2,Evening Ride,7.07,0 days 00:41:29,0 days 00:44:25,167.98,Ride,Ride,None,14606591445,...,20:11:00,Monday,0,May,5,2025-05-01,2025-05-01,May 2025,2025,2025-05-31T21:31:00
3,2,At the farm,4.21,0 days 00:43:58,0 days 00:44:30,500.66,Run,Run,0.0,14592270138,...,10:57:00,Sunday,6,May,5,2025-05-01,2025-05-01,May 2025,2025,2025-05-31T21:31:00
4,2,Night Walk,1.62,0 days 00:31:43,0 days 00:31:43,135.17,Walk,Walk,None,14592494813,...,22:29:00,Saturday,5,May,5,2025-05-01,2025-05-01,May 2025,2025,2025-05-31T21:31:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,2,Evening cruise,2.04,0 days 00:18:30,0 days 00:18:30,101.05,Run,Run,0.0,3966133614,...,19:40:00,Tuesday,1,August,8,2020-08-01,2020-08-01,Aug 2020,2020,2025-05-31T21:31:00
639,2,Afternoon Hike,5.03,0 days 02:13:32,0 days 02:36:08,662.73,Hike,Hike,None,3951601790,...,14:41:00,Saturday,5,August,8,2020-08-01,2020-08-01,Aug 2020,2020,2025-05-31T21:31:00
640,2,Afternoon Run,2.06,0 days 00:20:39,0 days 00:20:42,218.83,Run,Run,None,3942958315,...,17:20:00,Thursday,3,August,8,2020-08-01,2020-08-01,Aug 2020,2020,2025-05-31T21:31:00
641,2,Stress Reliever,2.06,0 days 00:20:43,0 days 00:20:47,221.46,Run,Run,0.0,3933051776,...,17:13:00,Tuesday,1,August,8,2020-08-01,2020-08-01,Aug 2020,2020,2025-05-31T21:31:00
